In [6]:

import random
import math
import time
import tracemalloc
import sys

# Set seed for reproducibility
random.seed(42)

# Measure memory usage
def get_memory_usage():
    return tracemalloc.get_traced_memory()[1]  # Peak memory usage

# Generate synthetic patient data
def generate_patient_data(num_samples=1000):
    X = [[random.uniform(36, 40),   # Body temperature (°C)
          random.uniform(80, 180),  # Systolic Blood Pressure (mmHg)
          random.uniform(60, 120),  # Diastolic Blood Pressure (mmHg)
          random.uniform(60, 150)]  # Heart Rate (bpm)
         for _ in range(num_samples)]

    y = [classify_health_status(x) for x in X]  # Generate labels
    return X, y

# Classify health status based on synthetic thresholds
def classify_health_status(patient):
    temp, sys_bp, dia_bp, heart_rate = patient

    if temp < 37.5 and 90 <= sys_bp <= 140 and 60 <= dia_bp <= 90 and 60 <= heart_rate <= 100:
        return 0  # Healthy
    elif 37.5 <= temp < 39 or sys_bp < 90 or sys_bp > 160 or dia_bp < 60 or dia_bp > 100 or heart_rate < 50 or heart_rate > 120:
        return 1  # At Risk
    else:
        return 2  # Critical

# Start tracking memory
tracemalloc.start()

# Generate dataset
X_train, y_train = generate_patient_data(500)
X_test, y_test = generate_patient_data(100)

# Neural Network Parameters
input_size = 4   # Temp, BP (Sys/Dia), Heart Rate
hidden_size = 16  # Hidden layer neurons
output_size = 3  # Health Status (Healthy, At Risk, Critical)
learning_rate = 0.005
epochs = 20

# Initialize Weights and Biases
W1 = [[random.uniform(-1, 1) for _ in range(hidden_size)] for _ in range(input_size)]
b1 = [random.uniform(-1, 1) for _ in range(hidden_size)]
W2 = [[random.uniform(-1, 1) for _ in range(output_size)] for _ in range(hidden_size)]
b2 = [random.uniform(-1, 1) for _ in range(output_size)]

# Activation Functions
def relu(x):
    return max(0, x)

def softmax(X):
    max_x = max(X)  # Stabilization to prevent overflow
    exp_values = [math.exp(x - max_x) for x in X]  # Subtract max(X) before exponentiation
    sum_values = sum(exp_values)
    return [x / sum_values for x in exp_values]

# Forward Propagation
def forward(x):
    hidden = [relu(sum(x[i] * W1[i][j] for i in range(input_size)) + b1[j]) for j in range(hidden_size)]
    output = [sum(hidden[j] * W2[j][k] for j in range(hidden_size)) + b2[k] for k in range(output_size)]
    return softmax(output)

# Training Loop
start_time = time.time()
for epoch in range(epochs):
    for i in range(len(X_train)):
        x = X_train[i]
        y = y_train[i]
        output = forward(x)
        target = [1 if j == y else 0 for j in range(output_size)]
        error = [(output[j] - target[j]) for j in range(output_size)]
        for j in range(hidden_size):
            for k in range(output_size):
                W2[j][k] -= learning_rate * error[k]
            b2[k] -= learning_rate * error[k]
        for i in range(input_size):
            for j in range(hidden_size):
                W1[i][j] -= learning_rate * sum(error)
            b1[j] -= learning_rate * sum(error)
    print(f"Epoch {epoch+1} completed")
training_time = time.time() - start_time

# Test with a new patient sample
new_patient = [random.uniform(36, 40), random.uniform(80, 180), random.uniform(60, 120), random.uniform(60, 150)]
real_status = classify_health_status(new_patient)
predicted_probs = forward(new_patient)
predicted_status = predicted_probs.index(max(predicted_probs))

status_labels = ["Healthy", "At Risk", "Critical"]
print(f"Real Health Status: {status_labels[real_status]}")
print(f"Predicted Health Status: {status_labels[predicted_status]}")

# Evaluate Accuracy
correct_predictions = 0
for i in range(len(X_test)):
    real_status = y_test[i]
    predicted_probs = forward(X_test[i])
    predicted_status = predicted_probs.index(max(predicted_probs))
    if predicted_status == real_status:
        correct_predictions += 1

accuracy = correct_predictions / len(X_test) * 100

# Measure memory and latency
peak_memory = get_memory_usage()

print(f"Model Accuracy: {accuracy:.2f}%")
print(f"Training Latency: {training_time:.4f} seconds")
print(f"Peak Memory Usage: {peak_memory / 1024:.2f} KB")

# Stop memory tracking
tracemalloc.stop()

Epoch 1 completed
Epoch 2 completed
Epoch 3 completed
Epoch 4 completed
Epoch 5 completed
Epoch 6 completed
Epoch 7 completed
Epoch 8 completed
Epoch 9 completed
Epoch 10 completed
Epoch 11 completed
Epoch 12 completed
Epoch 13 completed
Epoch 14 completed
Epoch 15 completed
Epoch 16 completed
Epoch 17 completed
Epoch 18 completed
Epoch 19 completed
Epoch 20 completed
Real Health Status: At Risk
Predicted Health Status: At Risk
Model Accuracy: 68.00%
Training Latency: 8.4927 seconds
Peak Memory Usage: 192.95 KB
